# Movie Recommendation Based on Sentiment Analysis of Tweets

In [1]:
# Disshow the warnning massage
import warnings
warnings.filterwarnings('ignore')

## improt related packages

In [2]:
import pandas as pd
from tensorflow import keras
import emoji
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
from nltk import word_tokenize
from keras.preprocessing import sequence
import twitter
from sklearn.preprocessing import LabelEncoder

## Sentiment Analysis Model

### Donload related word index

In [3]:
imdb = keras.datasets.imdb
word_index = imdb.get_word_index()

word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

### Load the model

In [4]:
new_model = keras.models.load_model('my_model.h5')

### Define Normalization Function

In [5]:
def norm(input):
    a=[]
    a.append(input)
    emoji.emojize(a[0])
    new_a=[]
    stop_words = set(stopwords.words('english'))
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(a[0])
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    new_a.append(filtered_sentence)
    emoji.emojize(a[0])
    return new_a


### Def model use function

In [6]:
def senti_ana(comment):
    if(comment == ""):
        return -1
    test=[]
    words_lis = norm(comment)
    # for word in word_tokenize("the <UNK> richard <UNK> dog is <UNK> to <UNK> joan fontaine dog however when <UNK> bing crosby arrives in town to sell a record player to the emperor his dog is attacked by <UNK> dog after a revenge attack where <UNK> is <UNK> from town a <UNK> insists that <UNK> dog must confront dog so that she can overcome her <UNK> fears this is arranged and the dogs fall in love so do <UNK> and <UNK> the rest of the film passes by with romance and at the end <UNK> dog gives birth but who is the father br br the dog story is the very weak vehicle that is used to try and create a story between humans its a terrible storyline there are 3 main musical pieces all of which are rubbish bad songs and dreadful choreography its just an extremely boring film bing has too many words in each sentence and delivers them in an almost irritating manner its not funny ever but its meant to be bing and joan"):
    for word in words_lis[0]:
        try:   
            if (word_index[word]<=10000 and type(word_index[word])==int):
                test.append(word_index[word])
            else:
                test.append(0)
        except:
            test.append(0)
#     for word in words_lis:
#         if (word in word_index and word_index[word].values <= 10000):
#             test.append[word_index[word]]
#         else:
#             test.append(0)
    test = [test]
    test = np.array(test)
    test = keras.preprocessing.sequence.pad_sequences(test,
                                                      value=0,
                                                      padding='post',
                                                      maxlen=256)
    m = new_model.predict_classes(np.array(test))

    result = m.reshape(-1)
    
    return result[0]

In [7]:
# Test the model
res = senti_ana("this film was one that i have waited to see for some time i was glad to find it has been everything anticipated the writing of this film has been so finely crafted and <UNK> far beyond what is seen by the audience i found it amusing that so many people watching will not read between some very important lines but indeed if not the movie will make sense in a different way and is very brilliant the film has many stories and characters woven together around this one character <UNK> a man whom has rose from the streets amidst many <UNK> and become a very powerful criminal after spending some time in prison <UNK> finds a <UNK> in the justice system and through a disturbing turn of events is released only to find everything is not at all what it seems <UNK> finds himself going up against the higher realm of society and political <UNK> in order to make clear how important a man's word is and stands for a war begins as the street is in arms against <UNK> of wealth and corrupt power br br a build up to explosive and powerful non stop twists and turns this film will leave you <UNK> i found the cast of this movie to be outstanding and is not a movie to be ignored excellent go rent it today")
print(res)

1


## Friendship based recommandation

### Load related database

In [8]:
# Comment
comment = pd.read_csv("./data/text.csv")

In [9]:
comment = comment[['user_id', 'movie_name', 'text']]
comment

,user_id,movie_name,text
0,829900052697120768,The Thin Man,RT @dril: you see a man standing near a woman ...
1,428549119,The Thin Man,RT @dril: you see a man standing near a woman ...
2,55645319,The Thin Man,"Kyrie digs Bob Dylan, eh? He’s out here soundi..."
3,1253914943243632640,The Thin Man,I’m tired of seeing these thick boys starve th...
4,466927126,The Thin Man,RT @dril: you see a man standing near a woman ...
...,...,...,...
104521,18958504,Ghostbusters (1984 Original),RT @historicalpixss: The original Ghostbusters...
104522,1273160442534379521,Ghostbusters (1984 Original),RT @historicalpixss: The original Ghostbusters...
104523,1325941160,Ghostbusters (1984 Original),RT @til_feed: Original Ghostbusters movie set ...
104524,410543687,Ghostbusters (1984 Original),RT @Iearnhistory: The original Ghostbusters mo...


In [10]:
users = comment.user_id
users = users.to_list()

### Get friends

In [11]:
def oauth_login():
    # XXX: Go to  to create an app and get values
    # for these credentials that you'll need to provide in place of these
    # empty string values that are defined as placeholders.
    # See https://developer.twitter.com/en/docs/basics/authentication/overview/oauth
    # for more information on Twitter's OAuth implementation.
    
    CONSUMER_KEY = 'RE2dHLBiiLVJqVO4YkkPl7eW4'
    CONSUMER_SECRET = '88pR1VmOqaFjIm5S2Nk22fL9qJ48l4zgOZ6QVxW4UbQeYg3duz'
    OAUTH_TOKEN = '1105923679232446469-LgyshrZzLfPO8YoOnFHhk33LTeiz2h'
    OAUTH_TOKEN_SECRET = 'nlbLt3wjx1nADK2sUD8G3x4HFSJoLj0uywKhErj6o9YqT'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

def get_friends(id_):
    twitter_api = oauth_login()
    print('authed')
    return twitter_api.friends.list(user_id=id_)

### define recommand function

In [13]:
def friend_recommand(userid, comment):
    fri_all = get_friends(userid)
    IDS = []
    for i in fri_all['users']:
        IDS.append(i['id'])
    friend = [i for i in IDS if i in users]
    movies = pd.DataFrame()
    for f in friend:
        res = comment.loc[comment['user_id']==f]
        movies = pd.concat([movies,res],axis=0,ignore_index=True)
    movies_watched = comment.loc[comment['user_id']==userid]['movie_name'].to_list()
    movies = movies[~movies['movie_name'].isin(movies_watched)]
    movies['senti'] = movies['text'].apply(senti_ana)
    rec_movies = movies['movie_name'][movies['senti']==1].to_list()
    return rec_movies

fri_movies = friend_recommand(428549119, comment)

authed


In [311]:
fri_movies

['The Vast of Night', 'Paddington 2']

## User based recommandation

### Load related database

In [14]:
# Comment
comment = pd.read_csv("./data/text.csv")
comment = comment[['user_id', 'movie_name', 'text']]
comment

,user_id,movie_name,text
0,829900052697120768,The Thin Man,RT @dril: you see a man standing near a woman ...
1,428549119,The Thin Man,RT @dril: you see a man standing near a woman ...
2,55645319,The Thin Man,"Kyrie digs Bob Dylan, eh? He’s out here soundi..."
3,1253914943243632640,The Thin Man,I’m tired of seeing these thick boys starve th...
4,466927126,The Thin Man,RT @dril: you see a man standing near a woman ...
...,...,...,...
104521,18958504,Ghostbusters (1984 Original),RT @historicalpixss: The original Ghostbusters...
104522,1273160442534379521,Ghostbusters (1984 Original),RT @historicalpixss: The original Ghostbusters...
104523,1325941160,Ghostbusters (1984 Original),RT @til_feed: Original Ghostbusters movie set ...
104524,410543687,Ghostbusters (1984 Original),RT @Iearnhistory: The original Ghostbusters mo...


In [15]:
# Users
user = pd.read_csv("./data/user.csv")

In [16]:
user = user[['id', 'description', 'protected', 'followers_count', 'friends_count', 'listed_count', 'created_at', 'favourites_count', 'verified', 'statuses_count', 'profile_background_color', 'profile_background_tile', 'profile_link_color', 'profile_sidebar_border_color', 'profile_sidebar_fill_color', 'profile_text_color']]
user

,id,description,protected,followers_count,friends_count,listed_count,created_at,favourites_count,verified,statuses_count,profile_background_color,profile_background_tile,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color
0,829900052697120768,i AM the heterosexual stop telling me no//hate...,False,111,143,6,Fri Feb 10 03:49:26 +0000 2017,54142,False,8526,F5F8FA,False,1DA1F2,C0DEED,DDEEF6,333333
1,428549119,Antifa posting associate \nenby dale gribble,False,169,793,0,Sun Dec 04 22:12:59 +0000 2011,66158,False,13655,000000,False,E81C4F,000000,000000,000000
2,55645319,NaN,False,249,1060,2,Fri Jul 10 19:40:03 +0000 2009,1710,False,1095,C0DEED,False,1DA1F2,C0DEED,DDEEF6,333333
3,1253914943243632640,♊️♌️♐️Judge yo Mama before judging me. I can’t...,False,1017,871,3,Sat Apr 25 05:13:30 +0000 2020,9172,False,6852,F5F8FA,False,1DA1F2,C0DEED,DDEEF6,333333
4,466927126,mmm milk,False,293,255,1,Tue Jan 17 23:09:24 +0000 2012,4432,False,43151,C0DEED,True,0084B4,FFFFFF,DDEEF6,333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90203,18958504,Tweet lots so engage & converse | Prev @ebay 🛒...,False,9435,179,1290,Tue Jan 13 23:29:52 +0000 2009,152,True,156827,131516,True,FF691F,FFFFFF,000000,0E73AD
90204,1273160442534379521,grains of gold among sand 🧜🏻‍♀️🌙,False,32,690,0,Wed Jun 17 07:48:31 +0000 2020,3350,False,271,F5F8FA,False,1DA1F2,C0DEED,DDEEF6,333333
90205,1325941160,#BLM #IFB #IFR #Resister #FBR #F4F #Biden2020...,False,30698,30106,453,Thu Apr 04 02:29:02 +0000 2013,48559,False,41915,C0DEED,False,1DA1F2,C0DEED,DDEEF6,333333
90206,410543687,snot-nosed little punk.,False,350,334,0,Sat Nov 12 07:17:07 +0000 2011,481,False,14298,642D8B,True,E81C4F,FFFFFF,7AC3EE,3D1957


### Sentiment analysis on description

In [17]:
# user['description'] = user['description'].fillna("")
# user['description_senti'] = user['description'].apply(senti_ana)

In [18]:
# user.to_csv('./data/user_senti.csv')

In [19]:
def euclidean(p, q):
    # 如果两数据集数目不同，计算两者之间都对应有的数
    same = 0
    for i in p:
        if i in q:
            same += 1
 
    # 计算欧几里德距离,并将其标准化
    e = sum([(p[i] - q[i]) ** 2 for i in range(same)])
    return 1 / (1 + e ** .5)

In [20]:
def noramlization(data):
    minVals = data.min(0)
    maxVals = data.max(0)
    ranges = maxVals - minVals
    normData = np.zeros(np.shape(data))
    m = data.shape[0]
    normData = data - np.tile(minVals, (m, 1))
    normData = normData/np.tile(ranges, (m, 1))
    return normData

In [21]:
def user_sim_recommand(userid, comment, user):
    user_senti = pd.read_csv("./data/user_senti.csv")
    user[['senti']] = user_senti[['description_senti']]
    user = user[['id', 'description', 'protected', 'followers_count', 'friends_count',
       'listed_count', 'created_at', 'favourites_count', 'verified',
       'statuses_count', 'profile_background_color', 'profile_text_color',
       'senti']]
    le = LabelEncoder()
    le.fit(user[['verified']].values.tolist())
    user[['verified']]=le.transform(user[['verified']].values.tolist())
    le = LabelEncoder()
    le.fit(user[['profile_background_color']].values.tolist())
    user[['profile_background_color']]=le.transform(user[['profile_background_color']].values.tolist())
    le = LabelEncoder()
    le.fit(user[['profile_text_color']].values.tolist())
    user[['profile_text_color']]=le.transform(user[['profile_text_color']].values.tolist())
    le.fit(user[['protected']].values.tolist())
    user[['protected']]=le.transform(user[['protected']].values.tolist())
    targ = user[user['id']==userid]
    targ.drop(['id', 'description', 'created_at'],axis=1, inplace=True)
    targ =targ.values.tolist()
    targ = targ[0]
    ori_user = user.copy(deep=True)
    user.drop(['id', 'description', 'created_at'],axis=1, inplace=True)
    le = LabelEncoder()
    candi = user.values.tolist()
    similarity = []
    for i in candi:
        similarity.append(euclidean(targ, i))
    result = []
    for i in range(5):
        result.append(similarity.index(sorted(similarity)[-(i+2)]))
    resID = []
    for i in result:
        resID.append(int(ori_user[i-1:i].id.values))
    res_movies = []
    res_simi = []
    for i in range(5):
        movies = pd.DataFrame()
        res = comment.loc[comment['user_id']==resID[i]]
        movies = pd.concat([movies,res],axis=0,ignore_index=True)
        movies_watched = comment.loc[comment['user_id']==428549119]['movie_name'].to_list()
        movies = movies[~movies['movie_name'].isin(movies_watched)]
        movies['senti'] = movies['text'].apply(senti_ana)
        rec_movies = movies['movie_name'][movies['senti']==1].to_list()
        res_movies.append(rec_movies)
        res_simi.append(sorted(similarity)[i+1])
    res_simi = np.array(res_simi)
    res_simi_norm = []
    for i in res_simi:
        res_simi_norm.append(i/res_simi.sum())
    
    res_simi_norm.reverse()
    return res_movies, res_simi_norm

In [22]:
usr_movies, usr_weights = user_sim_recommand(428549119, comment, user)

In [25]:
usr_weights

[0.2541035146251795,
 0.20089949269078483,
 0.18767939367737027,
 0.1835083500878237,
 0.17380924891884164]

## Combine

In [26]:
def recommand_movie(userid, comment, user):
    fri_movies = friend_recommand(userid, comment)
    usr_movies, usr_weights = user_sim_recommand(userid, comment, user)
    
    movies = {}
    for i in fri_movies:
        movies[i]=0.5
    for i in range(len(usr_weights)):
        for j in usr_movies[i]:
            if j in movies.keys():
                movies[j]+=0.5*usr_weights[i]
            else:
                movies[j]=0.5*usr_weights[i]
    return movies

In [27]:
# Comment
comment = pd.read_csv("./data/text.csv")
comment = comment[['user_id', 'movie_name', 'text']]

# Users
user = pd.read_csv("./data/user.csv")
user = user[['id', 'description', 'protected', 'followers_count', 'friends_count', 'listed_count', 'created_at', 'favourites_count', 'verified', 'statuses_count', 'profile_background_color', 'profile_background_tile', 'profile_link_color', 'profile_sidebar_border_color', 'profile_sidebar_fill_color', 'profile_text_color']]

In [28]:
movies = recommand_movie(428549119, comment, user)

authed


In [29]:
n = 1
for key, value in movies.items():
    print("No. %d Movie name: %20s\tWeight: %3f" %(n, key, value))
    n+=1

No. 1 Movie name:    The Vast of Night	Weight: 0.500000
No. 2 Movie name:         Paddington 2	Weight: 0.500000
No. 3 Movie name:               Frozen	Weight: 0.127052
No. 4 Movie name:    Shaun of the Dead	Weight: 0.100450
No. 5 Movie name:         The Lunchbox	Weight: 0.093840
No. 6 Movie name:     The Lost Weekend	Weight: 0.091754
No. 7 Movie name:   Ernest & Célestine	Weight: 0.086905


In [308]:
from keras.utils import plot_model
plot_model(new_model, to_file='model.pdf',show_shapes=False)

In [310]:
movies

{'The Vast of Night': 0.5,
 'Paddington 2': 0.5,
 'Frozen': 0.15410699368699324,
 'Shaun of the Dead': 0.1309751972666996,
 'The Lunchbox': 0.076901778823475,
 'The Lost Weekend': 0.07318018921947222,
 'Ernest & Célestine': 0.06483584100335998}